In [1]:
import illustris_python as il
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

from holodeck import utils,cosmo
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC
from holodeck import _PATH_DATA
import os

from holodeck.discrete import population, evolution
from holodeck import hardening
import holodeck as holo

In [2]:
from holodeck import triples

## Comparing Illustris TNG galaxy merger and BH merger file

In [3]:
TNG100_merger_file_path = '/orange/lblecha/pranavsatheesh/illustris-mbh-data/L75n1820TNG/'
merger_file = h5py.File(TNG100_merger_file_path+'L75n1820TNG_blackhole-mergers.hdf5', 'r')
print(merger_file.keys())
print('blackhole-merger keys:',merger_file['blackhole-mergers'].keys())
print('subhalo keys:',merger_file['subhalos'].keys())
print('param keys:',merger_file['params'].keys())
print(merger_file['blackhole-mergers/bh_mass'].shape)

<KeysViewHDF5 ['blackhole-mergers', 'metadata', 'params', 'subhalos']>
blackhole-merger keys: <KeysViewHDF5 ['bh_mass', 'bhid', 'bhid_next_snap', 'fuzz_flag', 'pt_mass', 'scafa', 'snap', 'tree_next', 'tree_prev']>
subhalo keys: <KeysViewHDF5 ['SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloGasMetallicity', 'SubhaloHalfmassRadType', 'SubhaloLenType', 'SubhaloMassInHalfRadType', 'SubhaloMassType', 'SubhaloParent', 'SubhaloPos', 'SubhaloSFR', 'SubhaloSFRinHalfRad', 'SubhaloStarMetallicity', 'SubhaloStellarPhotometrics', 'SubhaloVelDisp', 'SubhaloVmax', 'SubhaloVmaxRad']>
param keys: <KeysViewHDF5 ['AGB_MassTransferOn', 'ActivePartFracForNewDomainDecomp', 'AdaptiveHydroSofteningSpacing', 'BlackHoleAccretionFactor', 'BlackHoleCenteringMassMultiplier', 'BlackHoleEddingtonFactor', 'BlackHoleFeedbackFactor', 'BlackHoleMaxAccretionRadius', 'BlackHoleRadiativeEfficiency', 'BoxSize', 'CellMaxAngleFactor', 'CellShapingSpeed', 'ComovingIntegrationOn', 'CoolingOn', 'CoolingTablePath', 'Couran

In [4]:
TNG100_galaxy_merger_file ='galaxy-mergers_TNG100-1_gas-100_dm-100_star-100_bh-001.hdf5'
fname = os.path.join(_PATH_DATA,TNG100_galaxy_merger_file)
galaxy_merger_file = h5py.File(fname, 'r')
print(galaxy_merger_file.keys())
print(galaxy_merger_file.attrs.keys())

<KeysViewHDF5 ['ProgMassRatio', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloGrNr', 'SubhaloHalfmassRadType', 'SubhaloLenType', 'SubhaloMassInHalfRadType', 'SubhaloMassInRadType', 'SubhaloMassType', 'SubhaloPos', 'SubhaloSFR', 'SubhaloVel', 'SubhaloVelDisp', 'fpMass', 'npMass', 'shids_subf', 'shids_tree', 'snaps', 'time']>
<KeysViewHDF5 ['HubbleParam', 'Omega0', 'OmegaLambda', 'box_volume_mpc', 'created', 'merger_components_in_data_arrays', 'min_parts', 'num_mergers', 'part_names', 'part_types', 'snaptimes']>


### Find co-existing binaries

In [5]:
TNG100_merger_file_path = '/orange/lblecha/pranavsatheesh/illustris-mbh-data/L75n1820TNG/'
TNG100_merger_file_name = 'L75n1820TNG_blackhole-mergers.hdf5'

#creating a population object for TNG using the BH merger data
pop_TNG_bh = population.Pop_Illustris(basepath=TNG100_merger_file_path,fname=TNG100_merger_file_name,bh_merger_data_flag=True,min_BH_mass=5*1e6,min_DM=300,min_gas=100,min_star=100)

Filtering black holes with mass > 5.0e+06 Msun...
Applying particle cuts: N(DM) > 300, N(gas) > 100, N(star) > 100
Number of valid mergers:6291


In [6]:
fix_time = 3.0 * GYR

fixed_TNG_bh = hardening.Fixed_Time_2PL.from_pop(pop_TNG_bh, fix_time)
evo_TNG_bh = evolution.Evolution(pop_TNG_bh, fixed_TNG_bh)
evo_TNG_bh.evolve(progress=True)

evolving binaries:   0%|          | 0/99 [00:00<?, ?it/s]

In [7]:
triples.Triples(pop_TNG_bh,evo_TNG_bh)

fraction of triple interactions 0.325862343029725


In [9]:
fix_time = 3.0 * GYR

fixed_TNG_bh = hardening.Fixed_Time_2PL.from_pop(pop_TNG_bh, fix_time)
evo_TNG_bh = evolution.Evolution(pop_TNG_bh, fixed_TNG_bh)
evo_TNG_bh.evolve(progress=True)

evolving binaries:   0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
overlap_counter=0
trip_idx_data = []

for i in range(len(TNG_trip.repeated_ids)):
    repeated_id_idxs = np.argwhere(TNG_trip.bhids == TNG_trip.repeated_ids[i])[:,0]
    for i in range(len(repeated_id_idxs)-1):
        first_binary_idx = repeated_id_idxs[i]
        second_binary_idx = repeated_id_idxs[i+1]

        first_binary_zform = TNG_trip.z_binary_form[first_binary_idx]
        second_binary_zform = TNG_trip.z_binary_form[second_binary_idx]

        first_binary_tlook_form = evo_TNG_bh.tlook[first_binary_idx][0]
        first_binary_tlook_merger = evo_TNG_bh.tlook[first_binary_idx][-1]

        second_binary_tlook_form = evo_TNG_bh.tlook[second_binary_idx][0]
        second_binary_tlook_merger = evo_TNG_bh.tlook[second_binary_idx][-1]

        if(first_binary_tlook_merger<second_binary_tlook_form):
            overlap_counter = overlap_counter + 1
            trip_idx_data.append([first_binary_idx,second_binary_idx])
        
            

print('fraction of triple interactions',overlap_counter/TNG_trip.N_binaries)

fraction of triple interactions 0.325862343029725


In [44]:
for i in range(len(TNG_trip.repeated_ids)):
    common_elements = np.intersect1d(TNG_trip.bhids[trip_idx_data[0][0]], TNG_trip.bhids[trip_idx_data[0][1]])

    if common_elements.size == 0:
        raise ValueError("No common elements found between the two arrays.")

In [37]:
TNG_trip.bhids[trip_idx_data[0][0]]

array([101364043651, 101888649851], dtype=uint64)

In [38]:
TNG_trip.bhids[trip_idx_data[0][1]]

array([101888649851, 107458968134], dtype=uint64)

In [25]:
overlap_counter/TNG_trip.N_binaries * 100

32.5862343029725

In [32]:
for i in range(len(repeated_id_idxs)-1):
    first_binary_idx = repeated_id_idxs[i]
    second_binary_idx = repeated_id_idxs[i+1]

    first_binary_zform = TNG_trip.z_binary_form[first_binary_idx]
    second_binary_zform = TNG_trip.z_binary_form[second_binary_idx]

    first_binary_tlook_form = evo_TNG_bh.tlook[first_binary_idx][0]
    first_binary_tlook_merger = evo_TNG_bh.tlook[first_binary_idx][-1]

    second_binary_tlook_form = evo_TNG_bh.tlook[second_binary_idx][0]
    second_binary_tlook_merger = evo_TNG_bh.tlook[second_binary_idx][-1]

    if(first_binary_tlook_merger<second_binary_tlook_form):
        print('first binary:',first_binary_idx)
        print('second binary:',second_binary_idx)
        print('first binary merger tlook:',first_binary_tlook_merger)
        print('second binary formation tlook:',second_binary_tlook_form)
        print('overlap')

first binary: 1
second binary: 108
first binary merger tlook: 3.025165970605022e+17
second binary formation tlook: 3.7040214284979923e+17
overlap
first binary: 108
second binary: 129
first binary merger tlook: 2.755567394669094e+17
second binary formation tlook: 3.6824438143007014e+17
overlap
first binary: 129
second binary: 182
first binary merger tlook: 2.733585287589266e+17
second binary formation tlook: 3.6314710573186656e+17
overlap


[[3932    1]
 [5709    0]]
repeated_id_idxs: [3991 4561]


first_binary_zform: 0.4684956290227602
second_binary_zform: 0.3432645358013582


In [33]:
first_binary_idx = repeated_id_idxs[0]
second_binary_idx = repeated_id_idxs[1]

first_binary_zform = z_binary_form[first_binary_idx]
second_binary_zform = z_binary_form[second_binary_idx]
print('first_binary_zform:',first_binary_zform)
print('second_binary_zform:',second_binary_zform)

first_binary_zform: 5.037808758445386
second_binary_zform: 4.136924728641951


In [37]:
holo.triples

AttributeError: module 'holodeck' has no attribute 'triples'

In [ ]:
#for each pair of overlapping binaries the first one that formed (higher z-form) is the first binary and the second one is the second binary

for i in range(len(repeated_id_idxs)):
    first_binary_idx = repeated_id_idxs[i]
    second_binary_idx = repeated_id_idxs[i+1]

0
1


In [59]:
z_binary_form[np.argwhere(bhids == repeated_ids[1])[:,0]]

array([5.03780876, 4.13692473, 3.82157752])

In [60]:
snap_binary[np.argwhere(bhids == repeated_ids[1])[:,0]]

array([17, 21, 22])

In [40]:
print(bhids[0],z_binary_form[0])

[106045114669 106066328519] 7.235943303766296


In [20]:
unique_ids, counts = np.unique(merger_file['blackhole-mergers/bhid'][valid_merger_flag], return_counts=True)
repeated_ids = unique_ids[counts > 1]
percentage_repeated = (len(repeated_ids) / len(unique_ids)) * 100
print(f"Percentage of repeated IDs: {percentage_repeated:.2f}%")

Percentage of repeated IDs: 28.62%


In [35]:
np.argwhere(merger_file['blackhole-mergers/bhid'][valid_merger_flag]==repeated_ids[0])

array([[ 3389,     1],
       [13786,     0]])

In [26]:
merger_file['blackhole-mergers/bhid'][valid_merger_flag][3389]

array([100014872824, 100018008124], dtype=uint64)

In [28]:
merger_file['blackhole-mergers/bhid'][valid_merger_flag][13786]

array([100018008124, 116499017021], dtype=uint64)

In [32]:
print(repeated_ids,counts[counts > 1])

[100018008124 100134256609 100260425143 ... 122274990870 122304857114
 122391728950] [2 3 2 ... 4 4 2]


#### Trying with galaxy merger files

In [2]:
merger_file_path = '/home/pranavsatheesh/mbhb_evolution/holodeck/holodeck/data/'
#merger_file_path = '/home/pranavsatheesh/host_galaxies/data/merger_files/'
merger_file_name = 'galaxy-mergers_TNG100-1_gas-100_dm-100_star-100_bh-001.hdf5'
#merger_file_name = 'galaxy-mergers_TNG50-1_gas-100_dm-100_star-100_bh-001.hdf5'

In [3]:
merger_file = h5py.File(merger_file_path+merger_file_name, 'r')
print(merger_file.keys())

<KeysViewHDF5 ['ProgMassRatio', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloGrNr', 'SubhaloHalfmassRadType', 'SubhaloLenType', 'SubhaloMassInHalfRadType', 'SubhaloMassInRadType', 'SubhaloMassType', 'SubhaloPos', 'SubhaloSFR', 'SubhaloVel', 'SubhaloVelDisp', 'fpMass', 'npMass', 'shids_subf', 'shids_tree', 'snaps', 'time']>


In [4]:
Nmergers = merger_file['shids_subf'][:,0].shape[0]
print(Nmergers)

3284


In [5]:
id_prog_1 = merger_file['shids_subf'][:,0]
id_prog_2 = merger_file['shids_subf'][:,1]
id_desc = merger_file['shids_subf'][:,2]

snap_prog_1 = merger_file['snaps'][:,0]
snap_prog_2 = merger_file['snaps'][:,1]
snap_desc = merger_file['snaps'][:,2]

## Trying with BH merger file

In [3]:
bh_merger_file_path = '/orange/lblecha/IllustrisTNG/Runs/TNG100-1/postprocessing/blackholes/'
bh_merger_file_name = 'blackhole_mergers.hdf5'
bh_details_file_name = 'blackhole_details.hdf5'

In [4]:
bh_merger_file = h5py.File(bh_merger_file_path+bh_merger_file_name, 'r')
bh_details_file = h5py.File(bh_merger_file_path+bh_details_file_name, 'r')

In [5]:
print(bh_merger_file.keys())
print(bh_details_file.keys())

<KeysViewHDF5 ['Header', 'details', 'id_in', 'id_out', 'mass_in', 'mass_out', 'snapshot', 'time', 'tree']>
<KeysViewHDF5 ['Header', 'cs', 'id', 'mass', 'mdot', 'rho', 'time', 'unique']>


In [6]:
print(bh_merger_file['id_in'][:],bh_merger_file['id_out'][:])
print(bh_merger_file['snapshot'][:])
print(np.shape(bh_merger_file['id_out']))

[106045114669 102169506266 106065016166 ... 116402391015 107629987793
 108320880313] [106066328519 106037958296 106066328519 ... 120020142194 112530943142
 114209919489]
[11 11 12 ... 99 99 99]
(18374,)


##### Files from Luke

In [70]:
bh_merger_file['id_in'][:]

array([106045114669, 102169506266, 106065016166, ..., 116402391015,
       107629987793, 108320880313], dtype=uint64)

In [71]:
merger_file['blackhole-mergers/bhid'][:,0]

array([106045114669, 102169506266, 106065016166, ..., 107629987793,
       107713844288, 108320880313], dtype=uint64)

In [74]:
# Assuming bh_merger_file['id_in'][:] and merger_file['blackhole-mergers/bhid'][:,0] are loaded as arrays
id_in = bh_merger_file['id_in'][:]
bhid = merger_file['blackhole-mergers/bhid'][:, 0]

# Find IDs in id_in but not in bhid
only_in_id_in = np.setdiff1d(id_in, bhid)

# Find IDs in bhid but not in id_in
only_in_bhid = np.setdiff1d(bhid, id_in)

# Find common IDs
common_ids = np.intersect1d(id_in, bhid)

# Print results
print(f"IDs only in 'id_in': {only_in_id_in}")
print(f"IDs only in 'bhid': {only_in_bhid}")

IDs only in 'id_in': []
IDs only in 'bhid': [100010834623 100011375978 100031148013 ... 121656137827 121816908248
 121943202827]


In [77]:
# Assuming bhid is already loaded as merger_file['blackhole-mergers/bhid'][:, 0]
bhid1 = merger_file['blackhole-mergers/bhid'][:, 0]
bhid2 = merger_file['blackhole-mergers/bhid'][:, 1]

# Find black holes that appear in both bhid1 and bhid2 arrays
common_bhs = np.intersect1d(bhid1, bhid2)

# Calculate statistics
total_unique_bhid1 = np.unique(bhid1).size
total_unique_bhid2 = np.unique(bhid2).size
percentage_in_both = (len(common_bhs) / np.unique(np.concatenate((bhid1, bhid2))).size) * 100

print(f"Number of BHs that appear in both arrays: {len(common_bhs)}")
print(f"Percentage of unique BHs that appear in both arrays: {percentage_in_both:.2f}%")

Number of BHs that appear in both arrays: 7076
Percentage of unique BHs that appear in both arrays: 22.85%


#### Linking BH merger file to subhalo data

We have the linked files but they lack progenitor data. Future work should need to link them - BRAHMA sims ? 

In [29]:
snapnum=11
offset_path = '/orange/lblecha/IllustrisTNG/Runs/TNG100-1/postprocessing/offsets/'
offset_number = 'offsets_0%d.hdf5'%(snapnum)

In [31]:
offset_file = h5py.File(offset_path+offset_number, 'r')
print(offset_file.keys())

<KeysViewHDF5 ['FileOffsets', 'Group', 'Subhalo']>


In [2]:
# # Specify the field you want to load
# fields = ['ParticleIDs']

# # Load the ParticleIDs
# data = il.snapshot.loadSubset(basePath, snapNum=11, partType=5, fields=fields)

### Checking the illustris MBHB file

In [12]:
mbhb_merger_file_ill = '/orange/lblecha/pranavsatheesh/mbhb_data/ill-1_blackhole_mergers_fixed.npz'
mbhb_mergers_ill = np.load(mbhb_merger_file_ill)
print(list(mbhb_mergers_ill.keys()))

['ontop', 'run', 'scales', 'masses', 'created', 'ids', 'num', 'filename', 'version', 's2m', 'm2s']


In [13]:
ids = mbhb_mergers_ill['ids']
unique_ids, counts = np.unique(ids, return_counts=True)
repeated_ids = unique_ids[counts > 1]
percentage_repeated = (len(repeated_ids) / len(unique_ids)) * 100
print(f"Percentage of repeated IDs: {percentage_repeated:.2f}%")

Percentage of repeated IDs: 33.01%


### The fiducial model file

In [14]:
mbhb_data_dir = '/orange/lblecha/pranavsatheesh/mbhb_data/'
fid_e6_file = mbhb_data_dir+'mbhb-evolution_fiducial_ecc-evo-0.6_lc-shm06.hdf5'
fid_e6 = h5py.File(fid_e6_file, 'r')
print(fid_e6.keys())
print("meta:",fid_e6['meta'].keys())
print("evolution:",fid_e6['evolution'].keys())

<KeysViewHDF5 ['evolution', 'meta']>
meta: <KeysViewHDF5 ['SubhaloMassInHalfRadType', 'SubhaloSFRinHalfRad', 'snapshot', 'subhalo_id']>
evolution: <KeysViewHDF5 ['dadt', 'dadt_df', 'dadt_gw', 'dadt_lc', 'dadt_vd', 'dedt', 'eccen', 'masses', 'mdot_eff', 'scales', 'sep', 'times']>
